In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

tensor([ 1,  2,  3,  2,  2,  3,  3,  3,  3,  4,  4,  4,  5,  5,  5,  6,  6,  6,
         7,  7,  7,  8,  8,  8,  9,  9,  9, 10, 10, 10])

In [10]:
data = [
    [True, True, False, True, True],
    [False, True, False, False, True],
    [True, True, False, True, False],
    [True, True, True, False, True],
    [True, False, False, False, True],
]

import numpy as np

np.append(np.array(data).flatten(), True)

array([ True,  True, False,  True,  True, False,  True, False, False,
        True,  True,  True, False,  True, False,  True,  True,  True,
       False,  True,  True, False, False, False,  True,  True])

In [2]:
from agent import DQNAgent
import asyncio
import websockets
import json

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from agent import DQNAgent
import numpy as np




In [19]:
masked = torch.tensor(
    [
        [
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            1.0000e-08,
            -6.6584e-02,
        ]
    ]
)

print(masked)

masked.argmax()

tensor([[ 1.0000e-08,  1.0000e-08,  1.0000e-08,  1.0000e-08,  1.0000e-08,
          1.0000e-08,  1.0000e-08,  1.0000e-08,  1.0000e-08,  1.0000e-08,
          1.0000e-08,  1.0000e-08,  1.0000e-08,  1.0000e-08,  1.0000e-08,
          1.0000e-08,  1.0000e-08,  1.0000e-08,  1.0000e-08,  1.0000e-08,
          1.0000e-08,  1.0000e-08,  1.0000e-08,  1.0000e-08,  1.0000e-08,
         -6.6584e-02]])


tensor(0)

In [29]:
a = torch.randn(1, 26)
print(a)
a.argmax().item()

tensor([[ 0.2232, -0.7235,  0.2286, -0.7711,  0.3015,  0.1390,  0.9150, -1.0261,
          2.9890, -1.2376,  0.1683, -0.8119, -0.3365, -0.1047, -0.4431,  1.6885,
          0.9360, -0.4886, -0.4614,  1.0058,  0.3598, -0.1589, -0.1577,  0.6644,
          0.4298, -1.4695]])


8